In [1]:
// configuration: --help

OVERVIEW: MLIR modular optimizer driver

Available Dialects: acc, affine, arm_neon, arm_sve, async, avx512, gpu, linalg, llvm, llvm_arm_neon, llvm_arm_sve, llvm_avx512, nvvm, omp, pdl, pdl_interp, quant, rise, rocdl, scf, sdbm, shape, spv, std, tensor, test, tosa, vector
USAGE: mlir-opt [options] <input file>

OPTIONS:

Color Options:

  --color                                               - Use colors in output (default=autodetect)

General options:

  --allow-unregistered-dialect                          - Allow operation with no registered dialects
  --mlir-disable-threading                              - Disabling multi-threading within MLIR
  --mlir-elide-elementsattrs-if-larger=<uint>           - Elide ElementsAttrs with "..." that have more elements than the given upper limit
  --mlir-pretty-debuginfo                               - Print pretty debug info in MLIR output
  --mlir-print-debuginfo                                - Print debug info in MLIR output
  --mlir-print-ele

In [5]:
// configuration: --test-ml-opt-pass


module @custom_patterns {
module @color_patterns {
  // match for operations which do not have a color attribute and add one
  func @matcher(%root : !pdl.operation) {
    pdl_interp.apply_constraint "no_color"(%root : !pdl.operation) -> ^pat, ^end

  ^pat:
    pdl_interp.record_match @rewriters::@add_color(%root : !pdl.operation) : benefit(1), loc([%root]) -> ^end

  ^end:
    pdl_interp.finalize
  }
  module @rewriters {
    func @add_color(%root : !pdl.operation) {
      pdl_interp.apply_rewrite "add_color" on %root
      pdl_interp.finalize
    }
  }
}
}

module @pdl_patterns {

  // fuseReduceMap
  // apply reduceSeq f init (mapSeq g array) -> apply reduceSeq compose(f,g) init array
  pdl.pattern @fuseReduceMap : benefit(2) {
    %mapResT = pdl.type
    %applyMapResT = pdl.type
    %reduceResT = pdl.type
    %applyReduceResT = pdl.type
    %g = pdl.input
    %array = pdl.input
    %f = pdl.input
    %init = pdl.input

    %map, %mapRes = pdl.operation "rise.mapSeq" -> %mapResT
    %mapApply, %mapApplyRes = pdl.operation "rise.apply"(%mapRes, %g, %array) -> %applyMapResT
    %reduce, %reduceRes = pdl.operation "rise.reduceSeq" -> %reduceResT
    %redApply, %redApplyRes = pdl.operation "rise.apply"(%reduceRes, %f, %init, %mapApplyRes) -> %applyReduceResT

    pdl.rewrite %redApply with "fuseReduceMap"(%f, %init, %g, %array, %map, %mapApply : !pdl.value, !pdl.value, !pdl.value, !pdl.value, !pdl.operation, !pdl.operation)

  }

}

module @ir {
func @rise_fun(%arg0: memref<6x6xf32>, %arg1: memref<6x6xf32>, %arg2: memref<6x6xf32>) {
  "rise.lowering_unit"() ( {
    %0 = "rise.in"(%arg1) {"ksc.color" = [0.5 : f32, 0.5 : f32, 0.5 : f32]} : (memref<6x6xf32>) -> !rise.array<6, array<6, scalar<f32>>>
    %1 = "rise.in"(%arg2) {"ksc.color" = [0.5 : f32, 0.5 : f32, 0.5 : f32]} : (memref<6x6xf32>) -> !rise.array<6, array<6, scalar<f32>>>
    %2 = "rise.transpose"() {m = #rise.nat<6>, n = #rise.nat<6>, t = #rise.scalar<f32>} : () -> !rise.fun<array<6, array<6, scalar<f32>>> -> array<6, array<6, scalar<f32>>>>
    %3 = "rise.apply"(%2, %1) {"ksc.color" = [0.5 : f32, 0.5 : f32, 0.5 : f32]} : (!rise.fun<array<6, array<6, scalar<f32>>> -> array<6, array<6, scalar<f32>>>>, !rise.array<6, array<6, scalar<f32>>>) -> !rise.array<6, array<6, scalar<f32>>>
    %4 = "rise.lambda"() ( {
    ^bb0(%arg3: !rise.array<6, scalar<f32>>):  // no predecessors
      %7 = "rise.lambda"() ( {
      ^bb0(%arg4: !rise.array<6, scalar<f32>>):  // no predecessors
        %10 = "rise.zip"() {n = #rise.nat<6>, s = #rise.scalar<f32>, t = #rise.scalar<f32>} : () -> !rise.fun<array<6, scalar<f32>> -> fun<array<6, scalar<f32>> -> array<6, tuple<scalar<f32>, scalar<f32>>>>>
        %11 = "rise.apply"(%10, %arg3, %arg4) {"ksc.color" = [0.6 : f32, 0.1 : f32, 0.8 : f32]} : (!rise.fun<array<6, scalar<f32>> -> fun<array<6, scalar<f32>> -> array<6, tuple<scalar<f32>, scalar<f32>>>>>, !rise.array<6, scalar<f32>>, !rise.array<6, scalar<f32>>) -> !rise.array<6, tuple<scalar<f32>, scalar<f32>>>
        %12 = "rise.lambda"() ( {
        ^bb0(%arg5: !rise.tuple<scalar<f32>, scalar<f32>>):  // no predecessors
          %19 = "rise.fst"() {s = #rise.scalar<f32>, t = #rise.scalar<f32>} : () -> !rise.fun<tuple<scalar<f32>, scalar<f32>> -> scalar<f32>>
          %20 = "rise.snd"() {s = #rise.scalar<f32>, t = #rise.scalar<f32>} : () -> !rise.fun<tuple<scalar<f32>, scalar<f32>> -> scalar<f32>>
          %21 = "rise.apply"(%19, %arg5) {"ksc.color" = [0.9 : f32, 0.6 : f32, 0.2 : f32]} : (!rise.fun<tuple<scalar<f32>, scalar<f32>> -> scalar<f32>>, !rise.tuple<scalar<f32>, scalar<f32>>) -> !rise.scalar<f32>
          %22 = "rise.apply"(%20, %arg5) {"ksc.color" = [0.5 : f32, 0.1 : f32, 0.9 : f32]} : (!rise.fun<tuple<scalar<f32>, scalar<f32>> -> scalar<f32>>, !rise.tuple<scalar<f32>, scalar<f32>>) -> !rise.scalar<f32>
          %23 = "rise.embed"(%21, %22) ( {
          ^bb0(%arg6: f32, %arg7: f32):  // no predecessors
            %24 = mulf %arg6, %arg7 {"ksc.color" = [0.5 : f32, 0.5 : f32, 0.5 : f32]} : f32
            "rise.return"(%24) : (f32) -> ()
          }) {"ksc.color" = [0.5 : f32, 0.5 : f32, 0.5 : f32]} : (!rise.scalar<f32>, !rise.scalar<f32>) -> !rise.scalar<f32>
          "rise.return"(%23) : (!rise.scalar<f32>) -> ()
        }) : () -> !rise.fun<tuple<scalar<f32>, scalar<f32>> -> scalar<f32>>
        %13 = "rise.mapSeq"() {n = #rise.nat<6>, s = #rise.tuple<scalar<f32>, scalar<f32>>, t = #rise.scalar<f32>} : () -> !rise.fun<fun<tuple<scalar<f32>, scalar<f32>> -> scalar<f32>> -> fun<array<6, tuple<scalar<f32>, scalar<f32>>> -> array<6, scalar<f32>>>>
        %14 = "rise.apply"(%13, %12, %11) {"ksc.color" = [0.9 : f32, 0.9 : f32, 0.0 : f32]} : (!rise.fun<fun<tuple<scalar<f32>, scalar<f32>> -> scalar<f32>> -> fun<array<6, tuple<scalar<f32>, scalar<f32>>> -> array<6, scalar<f32>>>>, !rise.fun<tuple<scalar<f32>, scalar<f32>> -> scalar<f32>>, !rise.array<6, tuple<scalar<f32>, scalar<f32>>>) -> !rise.array<6, scalar<f32>>
        %15 = "rise.lambda"() ( {
        ^bb0(%arg5: !rise.scalar<f32>, %arg6: !rise.scalar<f32>):  // no predecessors
          %19 = "rise.embed"(%arg5, %arg6) ( {
          ^bb0(%arg7: f32, %arg8: f32):  // no predecessors
            %20 = addf %arg7, %arg8 {"ksc.color" = [0.5 : f32, 0.5 : f32, 0.5 : f32]} : f32
            "rise.return"(%20) : (f32) -> ()
          }) {"ksc.color" = [0.5 : f32, 0.5 : f32, 0.5 : f32]} : (!rise.scalar<f32>, !rise.scalar<f32>) -> !rise.scalar<f32>
          "rise.return"(%19) : (!rise.scalar<f32>) -> ()
        }) : () -> !rise.fun<scalar<f32> -> fun<scalar<f32> -> scalar<f32>>>
        %16 = "rise.literal"() {literal = #rise.lit<0.000000, scalar<f32>>, "ksc.color" = [0.6 : f32, 0.6 : f32, 0.1 : f32]} : () -> !rise.scalar<f32>
        %17 = "rise.reduceSeq"() {n = #rise.nat<6>, s = #rise.scalar<f32>, t = #rise.scalar<f32>} : () -> !rise.fun<fun<scalar<f32> -> fun<scalar<f32> -> scalar<f32>>> -> fun<scalar<f32> -> fun<array<6, scalar<f32>> -> scalar<f32>>>>
        %18 = "rise.apply"(%17, %15, %16, %14) {"ksc.color" = [0.6 : f32, 0.6 : f32, 0.1 : f32]} : (!rise.fun<fun<scalar<f32> -> fun<scalar<f32> -> scalar<f32>>> -> fun<scalar<f32> -> fun<array<6, scalar<f32>> -> scalar<f32>>>>, !rise.fun<scalar<f32> -> fun<scalar<f32> -> scalar<f32>>>, !rise.scalar<f32>, !rise.array<6, scalar<f32>>) -> !rise.scalar<f32>
        "rise.return"(%18) : (!rise.scalar<f32>) -> ()
      }) : () -> !rise.fun<array<6, scalar<f32>> -> scalar<f32>>
      %8 = "rise.mapSeq"() {n = #rise.nat<6>, s = #rise.array<6, scalar<f32>>, t = #rise.scalar<f32>} : () -> !rise.fun<fun<array<6, scalar<f32>> -> scalar<f32>> -> fun<array<6, array<6, scalar<f32>>> -> array<6, scalar<f32>>>>
      %9 = "rise.apply"(%8, %7, %3) {"ksc.color" = [0.8 : f32, 0.6 : f32, 0.5 : f32]} : (!rise.fun<fun<array<6, scalar<f32>> -> scalar<f32>> -> fun<array<6, array<6, scalar<f32>>> -> array<6, scalar<f32>>>>, !rise.fun<array<6, scalar<f32>> -> scalar<f32>>, !rise.array<6, array<6, scalar<f32>>>) -> !rise.array<6, scalar<f32>>
      "rise.return"(%9) : (!rise.array<6, scalar<f32>>) -> ()
    }) : () -> !rise.fun<array<6, scalar<f32>> -> array<6, scalar<f32>>>
    %5 = "rise.mapSeq"() {n = #rise.nat<6>, s = #rise.array<6, scalar<f32>>, t = #rise.array<6, scalar<f32>>} : () -> !rise.fun<fun<array<6, scalar<f32>> -> array<6, scalar<f32>>> -> fun<array<6, array<6, scalar<f32>>> -> array<6, array<6, scalar<f32>>>>>
    %6 = "rise.apply"(%5, %4, %0) {"ksc.color" = [0.9 : f32, 0.9 : f32, 0.0 : f32]} : (!rise.fun<fun<array<6, scalar<f32>> -> array<6, scalar<f32>>> -> fun<array<6, array<6, scalar<f32>>> -> array<6, array<6, scalar<f32>>>>>, !rise.fun<array<6, scalar<f32>> -> array<6, scalar<f32>>>, !rise.array<6, array<6, scalar<f32>>>) -> !rise.array<6, array<6, scalar<f32>>>
    "rise.out"(%arg0, %6) : (memref<6x6xf32>, !rise.array<6, array<6, scalar<f32>>>) -> ()
    "rise.return"() : () -> ()
  }) : () -> ()
  return
}
}



module  {
  module @custom_patterns  {
  }
  module @ir  {
    func @rise_fun(%arg0: memref<6x6xf32>, %arg1: memref<6x6xf32>, %arg2: memref<6x6xf32>) {
      "rise.lowering_unit"() ( {
        %0 = "rise.in"(%arg1) {ksc.color = [5.000000e-01 : f32, 5.000000e-01 : f32, 5.000000e-01 : f32]} : (memref<6x6xf32>) -> !rise.array<6, array<6, scalar<f32>>>
        %1 = "rise.in"(%arg2) {ksc.color = [5.000000e-01 : f32, 5.000000e-01 : f32, 5.000000e-01 : f32]} : (memref<6x6xf32>) -> !rise.array<6, array<6, scalar<f32>>>
        %2 = "rise.transpose"() {m = #rise.nat<6>, n = #rise.nat<6>, t = #rise.scalar<f32>} : () -> !rise.fun<array<6, array<6, scalar<f32>>> -> array<6, array<6, scalar<f32>>>>
        %3 = "rise.apply"(%2, %1) {ksc.color = [5.000000e-01 : f32, 5.000000e-01 : f32, 5.000000e-01 : f32]} : (!rise.fun<array<6, array<6, scalar<f32>>> -> array<6, array<6, scalar<f32>>>>, !rise.array<6, array<6, scalar<f32>>>) -> !rise.array<6, array<6, scalar<f32>>>
        %4 = "rise.lambda"() ( {
 

out = (
mapSeq(λ(x0 : !rise.array<6, scalar<f32>> =>
    mapSeq(λ(x1 : !rise.array<6, scalar<f32>> =>
        reduceSeq(λ(x2 : !rise.scalar<f32>,x3 : !rise.scalar<f32> =>
            embed(e4 : f32,e5 : f32 => {
                std.addf(x4,x5)
                }, x2,x3))
        ,l(0.000000),mapSeq(λ(x6 : !rise.tuple<scalar<f32>, scalar<f32>> =>
            embed(e7 : f32,e8 : f32 => {
                std.mulf(x7,x8)
                }, fst(x6),snd(x6)))
        ,zip(x0,x1))))
    ,transpose(in)))
,in))

out = (
mapSeq(λ(x0 : !rise.array<6, scalar<f32>> =>
    mapSeq(λ(x1 : !rise.array<6, scalar<f32>> =>
        reduceSeq(λ(x2 : !rise.tuple<scalar<f32>, scalar<f32>>,x3 : !rise.scalar<f32> =>
            λ(x4 : !rise.scalar<f32>,x5 : !rise.scalar<f32> =>
                embed(e6 : f32,e7 : f32 => {
                    std.addf(x6,x7)
                    }, x4,x5))
            (λ(x8 : !rise.tuple<scalar<f32>, scalar<f32>> =>
                embed(e9 : f32,e10 : f32 => {
                   

In [6]:
// configuration: --pass-pipeline='module(func(convert-rise-to-imperative,canonicalize))'
_

module  {
  module @custom_patterns  {
  }
  module @ir  {
    func @rise_fun(%arg0: memref<6x6xf32>, %arg1: memref<6x6xf32>, %arg2: memref<6x6xf32>) {
      %cst = constant 0.000000e+00 : f32
      %c0 = constant 0 : index
      %c6 = constant 6 : index
      %c1 = constant 1 : index
      scf.for %arg3 = %c0 to %c6 step %c1 {
        scf.for %arg4 = %c0 to %c6 step %c1 {
          %0 = alloc() : memref<f32>
          store %cst, %0[] : memref<f32>
          scf.for %arg5 = %c0 to %c6 step %c1 {
            %2 = load %arg1[%arg3, %arg5] : memref<6x6xf32>
            %3 = load %arg2[%arg5, %arg4] : memref<6x6xf32>
            %4 = mulf %2, %3 {ksc.color = [5.000000e-01 : f32, 5.000000e-01 : f32, 5.000000e-01 : f32], rise.cost = 1 : i32} : f32
            %5 = load %0[] : memref<f32>
            %6 = addf %5, %4 {ksc.color = [5.000000e-01 : f32, 5.000000e-01 : f32, 5.000000e-01 : f32], rise.cost = 1 : i32} : f32
            store %6, %0[] : memref<f32>
          }
          %1 = load %0